In [3]:
import os
import sys
import time
import glob
import numpy as np
import torch
import utils
import logging
import argparse
import torch.nn as nn
import genotypes
import torch.utils
import torchvision.datasets as dset
import torch.backends.cudnn as cudnn

from torch.autograd import Variable
from model import NetworkCIFAR as Network

from print_flops import *



free_darts_archs=['SYNFLOW_oneshot_seed0','SYNFLOW_oneshot_seed1','SYNFLOW_oneshot_seed2','SYNFLOW_oneshot_seed3','SYNFLOW_oneshot_seed4','SYNFLOW_oneshot_seed5','SYNFLOW_oneshot_seed6','SYNFLOW_oneshot_seed7','SYNFLOW_oneshot_seed8','SYNFLOW_oneshot_seed9',
                 'SYNFLOW_oneshot_20cells_seed0','SYNFLOW_oneshot_20cells_seed1','SYNFLOW_oneshot_20cells_seed2','SYNFLOW_oneshot_20cells_seed3','SYNFLOW_oneshot_20cells_seed4','SYNFLOW_oneshot_20cells_seed5','SYNFLOW_oneshot_20cells_seed6','SYNFLOW_oneshot_20cells_seed7','SYNFLOW_oneshot_20cells_seed8','SYNFLOW_oneshot_20cells_seed9',
                 'SYNFLOW_ImageNet_oneshot_seed0','SYNFLOW_ImageNet_oneshot_seed1','SYNFLOW_ImageNet_oneshot_seed2','SYNFLOW_ImageNet_oneshot_seed3','SYNFLOW_ImageNet_oneshot_seed4','SYNFLOW_ImageNet_oneshot_seed5','SYNFLOW_ImageNet_oneshot_seed6','SYNFLOW_ImageNet_oneshot_seed7','SYNFLOW_ImageNet_oneshot_seed8','SYNFLOW_ImageNet_oneshot_seed9']


save_name = 'architectures_parameter_on_CIFAR'
utils.create_exp_dir(save_name, scripts_to_save=glob.glob('*.py'))

log_format = '%(asctime)s %(message)s'
logging.basicConfig(stream=sys.stdout, level=logging.INFO,
    format=log_format, datefmt='%m/%d %I:%M:%S %p')
fh = logging.FileHandler(os.path.join(save_name, 'log.txt'))
fh.setFormatter(logging.Formatter(log_format))
logging.getLogger().addHandler(fh)

Experiment dir : architectures_parameter_on_CIFAR


In [4]:
id_num=0

for arch_id in free_darts_archs:
    parser = argparse.ArgumentParser("cifar")
    parser.add_argument('--data', type=str, default='/home/mzhang3/Data/DARTS/darts-master/data', help='location of the data corpus')
    parser.add_argument('--batch_size', type=int, default=96, help='batch size')
    parser.add_argument('--learning_rate', type=float, default=0.025, help='init learning rate')
    parser.add_argument('--momentum', type=float, default=0.9, help='momentum')
    parser.add_argument('--weight_decay', type=float, default=3e-4, help='weight decay')
    parser.add_argument('--report_freq', type=float, default=50, help='report frequency')
    parser.add_argument('--gpu', type=int, default=0, help='gpu device id')
    parser.add_argument('--epochs', type=int, default=600, help='num of training epochs')
    parser.add_argument('--init_channels', type=int, default=36, help='num of init channels')
    parser.add_argument('--layers', type=int, default=20, help='total number of layers')
    parser.add_argument('--model_path', type=str, default='saved_models', help='path to save the model')
    parser.add_argument('--auxiliary', action='store_true', default=True, help='use auxiliary tower')
    parser.add_argument('--auxiliary_weight', type=float, default=0.4, help='weight for auxiliary loss')
    parser.add_argument('--cutout', action='store_true', default=True, help='use cutout')
    parser.add_argument('--cutout_length', type=int, default=16, help='cutout length')
    parser.add_argument('--drop_path_prob', type=float, default=0.2, help='drop path probability')
    parser.add_argument('--save', type=str, default='EXP', help='experiment name')
    parser.add_argument('--seed', type=int, default=0, help='random seed')
    parser.add_argument('--arch', type=str, default=arch_id, help='which architecture to use')
    parser.add_argument('--grad_clip', type=float, default=5, help='gradient clipping')
    args = parser.parse_args([])


    CIFAR_CLASSES = 10


    #SYNFLOW_oneshot_seed0
    #SYNFLOW_oneshot_20cells_seed9
    #SYNFLOW_ImageNet_oneshot_seed9

    if id_num%10==0:
        mean_flop=0
        mean_MACS=0
        mean_Params=0

    np.random.seed(args.seed)
    torch.cuda.set_device(args.gpu)
    cudnn.benchmark = True
    torch.manual_seed(args.seed)
    cudnn.enabled=True
    torch.cuda.manual_seed(args.seed)

    genotype = eval("genotypes.%s" % args.arch)
    model = Network(args.init_channels, CIFAR_CLASSES, args.layers, args.auxiliary, genotype)
    #model = model.cuda()

    model.drop_path_prob = args.drop_path_prob * 1 / args.epochs

    import copy
    shape=(1,3,32,32)
    model = copy.deepcopy( model )

    model = add_flops_counting_methods(model)
    model = model.cuda()
    model.eval()

    cache_inputs = torch.zeros(*shape).cuda()
    _ = model(cache_inputs)
    FLOPs = compute_average_flops_cost( model ) / 1e6

    #utils.count_parameters_in_MB(model)

    from thop import profile
    input = torch.randn(1,3,32,32).cuda()
    macs, params = profile(model, inputs=(input, ))
    
    mean_flop +=FLOPs
    mean_MACS +=macs / 1e6
    mean_Params +=params / 1e6
    
    
    
    logging.info('FLOPs of architecture-'+args.arch+' is '+str(FLOPs)+'M')
    logging.info('MACS of architecture-'+args.arch+' is '+str(macs / 1e6)+'M')
    logging.info('Params of architecture-'+args.arch+' is '+str(params / 1e6)+'M')
    
    
    if (id_num+1)%10==0:
        logging.info('mean FLOPs of architecture-'+args.arch[:-1]+' is '+str(mean_flop/10)+'M')
        logging.info('mean MACS of architecture-'+args.arch[:-1]+' is '+str(mean_MACS / 10)+'M')
        logging.info('mean Params of architecture-'+args.arch[:-1]+' is '+str(mean_Params / 10)+'M')
    
    id_num = id_num+1


108 108 36
108 144 36
144 144 36
144 144 36
144 144 36
144 144 36
144 144 72
144 288 72
288 288 72
288 288 72
288 288 72
288 288 72
288 288 72
288 288 144
288 576 144
576 576 144
576 576 144
576 576 144
576 576 144
576 576 144
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[WARN] Cannot find rule for <class 'operations.ReLUConvBN'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'operations.SepConv'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'operations.DilConv'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.co

--- Logging error ---
Traceback (most recent call last):
  File "/data/mzhang3/anaconda3/lib/python3.6/logging/__init__.py", line 998, in emit
    self.flush()
  File "/data/mzhang3/anaconda3/lib/python3.6/logging/__init__.py", line 978, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/data/mzhang3/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/data/mzhang3/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/data/mzhang3/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/data/mzhang3/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/data/mzhang3/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/data/mzhang3/anaconda3/lib/python3.6/site-p

144 144 36
144 144 72
144 288 72
288 288 72
288 288 72
288 288 72
288 288 72
288 288 72
288 288 144
288 576 144
576 576 144
576 576 144
576 576 144
576 576 144
576 576 144
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[WARN] Cannot find rule for <class 'operations.ReLUConvBN'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'operations.DilConv'>. Treat it as zero Macs and zero Params.
[INFO] Register count_avgpool() for <class 'torch.nn.modules.pooling.AvgPool2d'>.
[WARN] Cannot find rule for <class 'operations.SepConv'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.ModuleList'>. Treat it as zero Macs and ze

--- Logging error ---
Traceback (most recent call last):
  File "/data/mzhang3/anaconda3/lib/python3.6/logging/__init__.py", line 998, in emit
    self.flush()
  File "/data/mzhang3/anaconda3/lib/python3.6/logging/__init__.py", line 978, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/data/mzhang3/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/data/mzhang3/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/data/mzhang3/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/data/mzhang3/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/data/mzhang3/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/data/mzhang3/anaconda3/lib/python3.6/site-p

144 144 36
144 144 36
144 144 72
144 288 72
288 288 72
288 288 72
288 288 72
288 288 72
288 288 72
288 288 144
288 576 144
576 576 144
576 576 144
576 576 144
576 576 144
576 576 144
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[WARN] Cannot find rule for <class 'operations.ReLUConvBN'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[WARN] Cannot find rule for <class 'operations.SepConv'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'operations.DilConv'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.ModuleList'>. Treat it as zero Macs 

...

--- Logging error ---
Traceback (most recent call last):
  File "/data/mzhang3/anaconda3/lib/python3.6/logging/__init__.py", line 998, in emit
    self.flush()
  File "/data/mzhang3/anaconda3/lib/python3.6/logging/__init__.py", line 978, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/data/mzhang3/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/data/mzhang3/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/data/mzhang3/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/data/mzhang3/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/data/mzhang3/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/data/mzhang3/anaconda3/lib/python3.6/site-p

576 576 144
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[WARN] Cannot find rule for <class 'operations.ReLUConvBN'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'operations.SepConv'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'operations.DilConv'>. Treat it as zero Macs and zero Params.
[INFO] Register count_avgpool() for <class 'torch.nn.modules.pooling.AvgPool2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.ModuleList'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'model.Cell'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'operations.FactorizedRed

--- Logging error ---
Traceback (most recent call last):
  File "/data/mzhang3/anaconda3/lib/python3.6/logging/__init__.py", line 998, in emit
    self.flush()
  File "/data/mzhang3/anaconda3/lib/python3.6/logging/__init__.py", line 978, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/data/mzhang3/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/data/mzhang3/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/data/mzhang3/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/data/mzhang3/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/data/mzhang3/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/data/mzhang3/anaconda3/lib/python3.6/site-p

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[WARN] Cannot find rule for <class 'operations.ReLUConvBN'>. Treat it as zero Macs and zero Params.
[INFO] Register count_avgpool() for <class 'torch.nn.modules.pooling.AvgPool2d'>.
[WARN] Cannot find rule for <class 'operations.SepConv'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'operations.DilConv'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'operations.Identity'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.ModuleList'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'model.Cell'>. Treat it as z

--- Logging error ---
Traceback (most recent call last):
  File "/data/mzhang3/anaconda3/lib/python3.6/logging/__init__.py", line 998, in emit
    self.flush()
  File "/data/mzhang3/anaconda3/lib/python3.6/logging/__init__.py", line 978, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/data/mzhang3/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/data/mzhang3/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/data/mzhang3/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/data/mzhang3/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/data/mzhang3/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/data/mzhang3/anaconda3/lib/python3.6/site-p